##Import library & Load the dataset from gdrive

In [8]:
# Import all the necessary library & files!
import os
import numpy as np
import pandas as pd

from keras.models  import Sequential, Model
from keras.layers import Conv2D, Flatten, MaxPooling2D, Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img
from keras.callbacks import ModelCheckpoint, EarlyStopping

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Initialization

In [0]:
image_path = './gdrive/My Drive/24hackton/images'
model_path = './gdrive/My Drive/24hackton/models'
train_img_path = '{}/train'.format(image_path)
val_img_path = '{}/val'.format(image_path)

## Data augmentation

In [3]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        width_shift_range=0.1,
        height_shift_range=0.1,
        rotation_range=25,
        vertical_flip=True,
        fill_mode='reflect',
        data_format='channels_last',
        brightness_range=[0.5, 1.5],
        featurewise_center=True,
        featurewise_std_normalization=True)

val_datagen = ImageDataGenerator(
    rescale=1/255)

# train_generator = train_datagen.flow_from_directory(
#     train_img_path,
#     target_size=(224, 224),
#     batch_size=64,
#     class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
    val_img_path,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical')

Found 232 images belonging to 5 classes.


###Load the train images from folder.

In [0]:
def get_img(filepath, size):
    '''
    Load single image from file.
    :param filepath: Path of the image.
    :size: size to resize
    '''
    img = load_img(filepath, target_size=size)
    img = img_to_array(img)
    img = np.array(img)/255.0

    return img


images = []
lables = []

for category_id, class_label in enumerate(val_generator.class_indices):
    print(class_label)
    print(category_id)
    train_img_path = '{}/train/{}'.format(image_path, class_label)
    for filename in os.listdir(train_img_path):
        images.append(get_img(os.path.join(
            train_img_path, filename), (224, 224)))
        lables.append(category_id)


### Apply SMOTE on the training data

In [0]:
from imblearn.over_sampling import SMOTE
from collections import Counter
from keras.utils.np_utils import to_categorical

counter = Counter(lables)
print(counter)

images = np.array(images)
print(images.shape)
n_sample, b, c, d = images.shape
images = images.reshape(n_sample, b*c*d)

oversample = SMOTE()
X, y = oversample.fit_resample(images, lables)
new_n_sample= (X.shape)[0]

X = X.reshape(new_n_sample, b, c, d)
y = to_categorical(y, 5)

## Building of CNN Classification Model

In [0]:
from keras.applications.inception_v3 import InceptionV3 

def create_model(dropout= True):
  base_model = InceptionV3(weights='imagenet', include_top=False)

  # add a global spatial average pooling layer
  x = base_model.output
  x = GlobalAveragePooling2D()(x)

  # # let's add a fully-connected layer
  predictions = Dense(5, activation='softmax')(x)

  # this is the model we will train
  model = Model(inputs=base_model.input, outputs=predictions)
  # model.summary()

  # freeze all convolutional layers
  for layer in base_model.layers:
      layer.trainable = False

  return model

model = create_model()

# Save the best model with max val_acc using ModelCheckpoint
checkpoint1 = ModelCheckpoint("{}/model.h5".format(model_path), monitor='val_acc', verbose=3, save_best_only=True, mode='max')
callbacks_list = [checkpoint1]

# Define parameters
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

model.summary()

### Train of model.

In [10]:
history = model.fit_generator(
    train_datagen.flow(X, y, batch_size=32),
    epochs=10,
    steps_per_epoch=new_n_sample//32,
    validation_data=val_generator,
    validation_steps=val_generator.n//val_generator.batch_size,
    callbacks=callbacks_list) 

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:724: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


102/102 [==============================] - 290s 3s/step - loss: 0.9582 - acc: 0.6685 - val_loss: 0.4282 - val_acc: 0.8490

Epoch 00001: val_acc improved from -inf to 0.84896, saving model to ./gdrive/My Drive/24hackton/models/model.h5
Epoch 2/10
102/102 [==============================] - 284s 3s/step - loss: 0.5319 - acc: 0.8284 - val_loss: 0.3509 - val_acc: 0.8631

Epoch 00002: val_acc improved from 0.84896 to 0.86310, saving model to ./gdrive/My Drive/24hackton/models/model.h5
Epoch 3/10
102/102 [==============================] - 285s 3s/step - loss: 0.4389 - acc: 0.8637 - val_loss: 0.4105 - val_acc: 0.8095

Epoch 00003: val_acc did not improve from 0.86310
Epoch 4/10
102/102 [==============================] - 285s 3s/step - loss: 0.3805 - acc: 0.8793 - val_loss: 0.3294 - val_acc: 0.8810

Epoch 00004: val_acc improved from 0.86310 to 0.88095, saving model to ./gdrive/My Drive/24hackton/models/model.h5
Epoch 5/10
102/102 [==============================] - 288s 3s/step - loss: 0.3485 -